In [1]:
import os
import datetime
import calendar
import pandas as pd
import geopandas as gpd
import numpy as np
import scipy
from sklearn.impute import KNNImputer

In [2]:
start_date = '2020-03-01'
end_date = '2022-03-31'
N_neighbors_spatial_interpolation = 3

In [ ]:
ltlas_gdf = gpd.GeoDataFrame.from_file(os.path.join("gis", 'lad19.geojson'))
ltlas_gdf.head()

In [ ]:
data_dfs = []
for year in [2020, 2021, 2022]:
    for month in range(1, 13):
        if month < 10:
            month = "0%d" % month
        for file_name in os.listdir('healthcare_access/England/'):
            if file_name.startswith("%d%s" % (year, month)) and file_name.endswith('.xlsx'):
                data_df = pd.read_excel('healthcare_access/England/%s' % file_name)
                for idx, row in data_df.iterrows():
                    if row['Unnamed: 1'] == 'Region':
                        break
                data_df = pd.read_excel('healthcare_access/England/%s' % file_name, skiprows=idx+1)
                date = datetime.datetime.strptime(
                    "%d-%s" % (year, month), '%Y-%m')
                data_df['date'] = date
                data_df = data_df[['date', 'Trust Name', 'Code', 'G&A beds available']]
                data_df['G&A beds available'] = data_df['G&A beds available'].astype(int)
                data_dfs.append(data_df)
                break
England_df = pd.concat(data_dfs, axis=0, ignore_index=True)

# https://data.england.nhs.uk/dataset/ods-nhs-trusts-and-sites
# https://epiforecasts.io/covid19.nhs.data/articles/mapping_summary.html
mapping_df = pd.read_csv("healthcare_access/England/trust_ltla_mapping.csv")
England_df = pd.merge(England_df, mapping_df,
                      left_on="Code", right_on="trust_code", how="left")
England_df = England_df[['date', 'G&A beds available', "trust_code", 'geo_code', 'p_geo']]
England_df = England_df.rename(columns={'G&A beds available': 'available_beds', 'geo_code': 'district_id'})
England_df['available_beds'] = England_df['available_beds'] * England_df['p_geo']
England_df = England_df.groupby(["district_id", "date"])['available_beds'].sum().reset_index()
England_df = England_df.sort_values(["district_id", "date"]).reset_index()
England_df['date'] = England_df['date'].dt.strftime("%Y-%m-%d")
dates = England_df['date'].unique()
dates = dates[~pd.isnull(dates)]
districts_without_beds = England_df[England_df['date'].isnull()]
new_data = []
for idx, row in districts_without_beds.iterrows():
    for date in dates:
        record = {
            "date": date,
            "district_id": row.district_id,
            "available_beds": np.nan
        }
        new_data.append(record)

for date in dates:
    # Aylesbury Vale, Chiltern, South Bucks, Wycombe => Buckinghamshire
    # West Devon => to be imputed by spatial interpolation
    for district_id in ["E07000004", "E07000005", "E07000006", "E07000007", 'E07000047']:
        record = {
            "date": date,
            "district_id": district_id,
            "available_beds": np.nan
        }
        new_data.append(record)

# E06000058, E07000043, E07000046
for district_id in England_df['district_id'].unique():
    if len(dates) != len(England_df[England_df['district_id'] == district_id]['date'].unique()):
        for date in dates:
            if date not in England_df[England_df['district_id'] == district_id]['date'].unique():
                record = {
                    "date": date,
                    "district_id": district_id,
                    "available_beds": np.nan
                }
                new_data.append(record)

districts_without_beds_df = pd.DataFrame.from_records(new_data)
England_df = pd.concat(
    [England_df[~England_df['date'].isnull()], districts_without_beds_df], axis=0)
England_df = England_df.sort_values(['district_id', 'date']).reset_index()

# Aylesbury Vale, Chiltern, South Bucks, Wycombe => Buckinghamshire
England_df.loc[England_df['district_id'] == "E07000004", 'available_beds'] = England_df.loc[England_df['district_id'] == "E06000060", 'available_beds'].values
England_df.loc[England_df['district_id'] == "E07000005", 'available_beds'] = England_df.loc[England_df['district_id'] == "E06000060", 'available_beds'].values
England_df.loc[England_df['district_id'] == "E07000006", 'available_beds'] = England_df.loc[England_df['district_id'] == "E06000060", 'available_beds'].values
England_df.loc[England_df['district_id'] == "E07000007", 'available_beds'] = England_df.loc[England_df['district_id'] == "E06000060", 'available_beds'].values

# Temporal interpolation for E06000058, E07000043, E07000046
England_df.loc[England_df['district_id'] == "E06000058", 'available_beds'] = England_df.loc[England_df['district_id'] == "E06000058", 'available_beds'].bfill()
England_df.loc[England_df['district_id'] == "E07000043", 'available_beds'] = England_df.loc[England_df['district_id'] == "E07000043", 'available_beds'].bfill()
England_df.loc[England_df['district_id'] == "E07000046", 'available_beds'] = England_df.loc[England_df['district_id'] == "E07000046", 'available_beds'].bfill()

England_df = pd.merge(England_df, ltlas_gdf, left_on=[
                      "district_id"], right_on=["district_id"], how="right")
England_df = England_df[['district_name', 'district_id', 'date', 'available_beds', 'district_lon', 'district_lat']]
England_df = England_df[England_df['district_id'].str.startswith("E")]
England_df = England_df[~England_df['date'].isnull()]

# Spatial interpolation for West Devon
def spatial_distance(X1, X2, missing_values=None):
    # X: ..., 'district_lon', 'district_lat'
    return scipy.spatial.distance.euclidean(X1[-2:], X2[-2:])

for idx, date in enumerate(dates):
    data_matrix = England_df[England_df['date'] == date][['available_beds', 'district_lon', 'district_lat']]
    imputer = KNNImputer(n_neighbors=N_neighbors_spatial_interpolation, weights='distance', metric=spatial_distance)
    imputed_matrix = imputer.fit_transform(data_matrix)
    imputed_df = pd.DataFrame(data=imputed_matrix[:, 0:len(
        ['available_beds'])], index=data_matrix.index, columns=['available_beds'])
    England_df.loc[(England_df['date'] == date), ['available_beds']] = England_df.loc[(
        England_df['date'] == date), ['available_beds']].fillna(value=imputed_df)

England_df = England_df.drop('district_lon', axis=1)
England_df = England_df.drop('district_lat', axis=1)

England_df['available_beds'] = England_df['available_beds'].astype(int)
England_df.head()

In [ ]:
data_df = pd.read_csv("healthcare_access/Northern Ireland/hs-inpatient-hts-tables-21-22.csv", skiprows=2)
data_df = data_df[data_df['Programme of Care'] == "Acute Services"]
data_df = data_df[['Quarter Ending', 'HSC Trust',
                   'Hospital', 'Specialty', 'Average Available Beds']]
data_df['Total Available Beds'] = data_df['Average Available Beds'].astype(int)
data_df = data_df.groupby(['Quarter Ending', 'HSC Trust'])[
    'Average Available Beds'].sum().reset_index()
data_df['Quarter Ending'] = pd.to_datetime(
    data_df['Quarter Ending'], format="%d/%m/%Y")
data_df = data_df.sort_values(['Quarter Ending', 'HSC Trust']).reset_index()
data_df = data_df.drop('index', axis=1)
data_df = data_df.rename(columns={"Quarter Ending": "date"})

new_data = []
for idx, row in data_df.iterrows():
    date_obj = row.date
    quarter = date_obj.quarter
    year = date_obj.year
    if quarter == 1:
        month_start = 1
    elif quarter == 2:
        month_start = 4
    elif quarter == 3:
        month_start = 7
    elif quarter == 4:
        month_start = 10
    hsc_trust = row['HSC Trust']
    beds = row['Average Available Beds']
    for month in range(month_start, month_start+3):
        new_data.append({
            "date": "%s-%s" % (year, month),
            "HSC Trust": hsc_trust,
            "Average Available Beds": int(beds)
        })
NI_df = pd.DataFrame.from_records(new_data)
NI_df['date'] = pd.to_datetime(NI_df['date'], format="%Y-%m")
NI_df = NI_df[(NI_df['date'] >= start_date) & (NI_df['date'] <= end_date)]
NI_df['date'] = NI_df['date'].dt.strftime("%Y-%m-%d")
NI_df = NI_df.reset_index().drop('index', axis=1)

# https://www.nisra.gov.uk/sites/nisra.gov.uk/files/publications/Maps-of-NI-Health-Trusts-and-Local-Government-Districts.pdf
NI_health_trust_to_local_authority = {
    "Belfast": ["N09000003", "N09000007"],
    "Northern": ["N09000001", "N09000004", "N09000008", "N09000009"],
    "South Eastern": ["N09000011", "N09000007", "N09000010"],
    "Southern": ["N09000002", "N09000010"],
    "Western": ["N09000005", "N09000006"]
}
new_data = []
for idx, row in NI_df.iterrows():
    hsc_trust = row['HSC Trust']
    for district_id in NI_health_trust_to_local_authority[hsc_trust]:
        record = {
            "date": row.date,
            "district_id": district_id,
            'available_beds': row['Average Available Beds']
        }
        new_data.append(record)
NI_df = pd.DataFrame.from_records(new_data)
NI_df = NI_df.groupby(['district_id', 'date'])[
    'available_beds'].sum().reset_index()
NI_df = pd.merge(NI_df, ltlas_gdf, left_on="district_id",
                 right_on="district_id", how="left")
NI_df = NI_df[['district_name', 'district_id', 'date', 'available_beds']]
NI_df['available_beds'] = NI_df['available_beds'].astype(int)
NI_df.head()

In [ ]:
data_df = pd.read_csv("healthcare_access/Scotland/Beds by Health Board of Treatment and Specialty.csv")
data_df = data_df[['FinancialYear', 'HB', 'Specialty',
                   'SpecialtyQF', 'AverageAvailableStaffedBeds']]
data_df = data_df[data_df['FinancialYear'] == '2020/21']
data_df = data_df[~(data_df['SpecialtyQF'] == 'z')]
data_df = data_df.groupby(['FinancialYear', 'HB'])['AverageAvailableStaffedBeds'].sum().reset_index()
new_data = []
for idx, row in data_df.iterrows():
    health_board = row['HB']
    beds = row['AverageAvailableStaffedBeds']
    for year in [2020, 2021, 2022]:
        for month in range(1, 13):
            new_data.append({
                "date": "%s-%s" % (year, month),
                "HB": health_board,
                "AverageAvailableStaffedBeds": int(beds)
            })
Scotland_df = pd.DataFrame.from_records(new_data)
Scotland_df['date'] = pd.to_datetime(Scotland_df['date'], format="%Y-%m")
Scotland_df = Scotland_df[(Scotland_df['date'] >= start_date) & (
    Scotland_df['date'] <= end_date)]
Scotland_df = Scotland_df.reset_index().drop('index', axis=1)
hb_name = pd.read_csv("healthcare_access/Scotland/Health Board 2014 - Health Board 2019.csv")
Scotland_df = pd.merge(Scotland_df, hb_name, left_on="HB",
                       right_on="HB", how="left")
Scotland_df = Scotland_df[['date', 'HB',
                           'HBName', 'AverageAvailableStaffedBeds']]
Scotland_df['HB'] = Scotland_df['HB'].str.strip()
Scotland_df.head()

# https://maps.gov.scot/ATOM/shapefiles/SG_NHS_HealthBoards_2019.zip
Scotland_HB_shape = gpd.GeoDataFrame.from_file('gis/Scotland_NHS_HealthBoards_2019/SG_NHS_HealthBoards_2019.shp')
Scotland_df = pd.merge(Scotland_df, Scotland_HB_shape, left_on=['HB'], right_on=['HBCode'], how="left")
Scotland_df = Scotland_df[['date', 'AverageAvailableStaffedBeds', 'geometry']]
Scotland_df = gpd.GeoDataFrame(Scotland_df)
Scoland_ltlas_gdf = ltlas_gdf[ltlas_gdf['district_id'].str.startswith('S')]
Scotland_df = Scotland_df.to_crs(Scoland_ltlas_gdf.crs)
Scotland_df = gpd.tools.sjoin(
    Scotland_df, Scoland_ltlas_gdf, predicate="intersects", how='right')
Scotland_df = Scotland_df[['district_name',
                           'district_id', 'date', 'AverageAvailableStaffedBeds']]
Scotland_df = Scotland_df.rename(
    columns={'AverageAvailableStaffedBeds': 'available_beds'})
Scotland_df = Scotland_df.groupby(['district_name', 'district_id', 'date'])[
    'available_beds'].sum().reset_index()
Scotland_df['available_beds'] = Scotland_df['available_beds'].astype(int)
Scotland_df['date'] = Scotland_df['date'].dt.strftime("%Y-%m-%d")
Scotland_df = Scotland_df[Scotland_df['district_id'].isin(
    Scoland_ltlas_gdf['district_id'].unique())]
Scotland_df.head()

In [ ]:
data_df = pd.read_csv("healthcare_access/Wales/NHS beds by organisation and site_clean.csv", na_values=".")
data_df = data_df[['HB', '2020-21 (9)']]
data_df = data_df.dropna()
data_df = data_df[data_df['HB'] != 'Velindre NHS Trust']
new_data = []
for idx, row in data_df.iterrows():
    health_board = row['HB']
    beds = row['2020-21 (9)']
    for year in [2020, 2021, 2022]:
        for month in range(1, 13):
            new_data.append({
                "date": "%s-%s" % (year, month),
                "HB": health_board,
                "AverageAvailableBeds": int(beds)
            })
Wales_df = pd.DataFrame.from_records(new_data)
Wales_df['date'] = pd.to_datetime(Wales_df['date'], format="%Y-%m")
Wales_df = Wales_df[(Wales_df['date'] >= start_date)
                    & (Wales_df['date'] <= end_date)]
Wales_df = Wales_df.reset_index().drop('index', axis=1)
Wales_df = Wales_df[['date', 'HB', 'AverageAvailableBeds']]
Wales_df['HB'] = Wales_df['HB'].str.strip()
Wales_df.head()

# https://www.data.gov.uk/dataset/6b0d997a-4888-4c59-abf4-e1dca4b149fb/local-health-boards-april-2019-boundaries-wa-bfc
Wales_HB_shape = gpd.GeoDataFrame.from_file('gis/Wales_NHS_HealthBoards_2019/Local_Health_Boards_(April_2019)_Boundaries_WA_BFC.shp')
Wales_df = pd.merge(Wales_df, Wales_HB_shape, left_on=[
                    'HB'], right_on=['lhb19nm'], how="left")
Wales_df = Wales_df[['date', 'AverageAvailableBeds', 'geometry']]
Wales_df = gpd.GeoDataFrame(Wales_df)
Wales_ltlas_gdf = ltlas_gdf[ltlas_gdf['district_id'].str.startswith("W")]
Wales_df = Wales_df.to_crs(Wales_ltlas_gdf.crs)
Wales_df = gpd.tools.sjoin(
    Wales_df, Wales_ltlas_gdf, predicate="intersects", how='right')
Wales_df = Wales_df[['district_name',
                     'district_id', 'date', 'AverageAvailableBeds']]
Wales_df = Wales_df.rename(columns={'AverageAvailableBeds': 'available_beds'})
Wales_df = Wales_df.groupby(['district_name', 'district_id', 'date'])[
    'available_beds'].sum().reset_index()
Wales_df['available_beds'] = Wales_df['available_beds'].astype(int)
Wales_df['date'] = Wales_df['date'].dt.strftime("%Y-%m-%d")
Wales_df = Wales_df[Wales_df['district_id'].isin(
    Wales_ltlas_gdf['district_id'].unique())]
Wales_df.head()

In [ ]:
uk_df = pd.concat([England_df, NI_df, Scotland_df, Wales_df],
                  axis=0, ignore_index=True)
uk_df = uk_df.sort_values(['district_name', 'date']).reset_index().drop('index', axis=1)
uk_df = uk_df.rename(columns={"available_beds": "no_of_available_beds"})
uk_df.head()

In [9]:
assert(len(ltlas_gdf['district_id'].unique())==len(uk_df['district_id'].unique()))

In [10]:
uk_df.to_csv("../health_care_access.csv", na_rep="N/A", index=False)